In [4]:
import pandas as pd
from scipy.io import loadmat
import numpy as np
import scipy.stats as s
from sklearn.preprocessing import StandardScaler

In [5]:
TrainData = pd.read_csv('datasets/train.csv')


In [6]:
sc = StandardScaler()

X_Train_Data = TrainData.iloc[:,1:]
Y_Train_Data = TrainData.iloc[:,0]
# X_Train_Data = sc.fit_transform(X_Train_Data

TestData = pd.read_csv('datasets/test.csv')
X_Test_Data = TestData.iloc[:,:]
Y_Test_Data = TestData.iloc[:,0]
# X_Test_Data = sc.fit_transform(X_Test_Data)



In [35]:
def PCA(Train_Data,Test_Data):
    Cov = np.cov(Train_Data.T)
    eigen_pairs = {}
    eigen_vals,eigen_vecs = np.linalg.eig(Cov)

    for i in range(len(eigen_vals)):
        eigen_pairs[eigen_vals[i]] = eigen_vecs[i] 

    tot = sum(eigen_vals)

    var_exp = [(i/tot) for i in sorted(eigen_vals,reverse= True)]
    cum_var_exp = np.cumsum(var_exp)


    for i in range(len(cum_var_exp)):
        if cum_var_exp[i] >= 0.95:
            k = i
            break;

    print("K :",k)
    print("LEN: ",len(eigen_vals))


    sorted(eigen_vals,reverse=True)


    w = []
    for i in eigen_vals:
        w.append(np.real(eigen_pairs[i][:k]))

    w = np.array(w)


    X_Train_Data_PCA = Train_Data.dot(w)
#     print("X_Train_Data_PCA.shape",X_Train_Data_PCA.shape)

    X_Test_Data_PCA = Test_Data.dot(w)
#     print("X_Test_Data_PCA.shape",X_Test_Data_PCA.shape)
    
    return X_Train_Data_PCA,X_Test_Data_PCA

In [36]:
PCapList =[Y_Train_Data[Y_Train_Data == i].shape[0]/60000 for i in range(10)]
PCapList


[0.06886666666666667,
 0.07806666666666667,
 0.06961666666666666,
 0.07251666666666666,
 0.06786666666666667,
 0.06325,
 0.06895,
 0.07335,
 0.06771666666666666,
 0.0698]

In [37]:
NumberList = {}
for i in range(10):
    NumberList[i] = X_Train_Data[Y_Train_Data == i]
    print(NumberList[i].shape)

(4132, 784)
(4684, 784)
(4177, 784)
(4351, 784)
(4072, 784)
(3795, 784)
(4137, 784)
(4401, 784)
(4063, 784)
(4188, 784)


In [38]:
def calProb(Test_Data_PCA,Pcap,Mean,cov):
    return Pcap * s.multivariate_normal.pdf(Test_Data_PCA,Mean,cov)

ProbList = []
for i in range(10):
    X_Train_Data_PCA,X_Test_Data_PCA = PCA(NumberList[i],X_Test_Data)
    Mean = np.mean(a = X_Train_Data_PCA,axis=0)
    Cov = np.cov(X_Train_Data_PCA.T)
    ProbList.append(calProb(X_Test_Data_PCA,PCapList[i],Mean,Cov))
    
    



K : 113
LEN:  784
K : 69
LEN:  784
K : 140
LEN:  784
K : 135
LEN:  784
K : 132
LEN:  784
K : 128
LEN:  784
K : 111
LEN:  784
K : 119
LEN:  784
K : 134
LEN:  784
K : 110
LEN:  784


In [44]:
probList=np.array(ProbList)
probList.shape

(10, 28000)

In [46]:
r=np.argmax(probList[1,:])
r

601

In [10]:
  
    
Predicted = []
All_Tot = sum(ProbList)
print(All_Tot)
Pred_Prob = []
for i in range(10):
    Pred_Prob.append(ProbList[i]/All_Tot)

Pred_Prob = np.array(Pred_Prob)
# print(Pred_Prob)

for j in range(len(X_Test_Data_PCA)):
    tslist = list(Pred_Prob[:,j])
#     print(sum(tslist))
#     print(tslist)
    idx = tslist.index(max(tslist))
#     print(idx," : ", Y_Test_Data[j] )
    Predicted.append(idx)
    
count = 0
for i in range(len(Y_Test_Data)):
    if Predicted[i] == Y_Test_Data[i]:
        count += 1
# print("Accuracy: ",(count/len(Y_Test_Data))*100)

[0.00000000e+000 3.06993295e-268 3.00367383e-220 ... 2.97264706e-275
 1.19234983e-241 0.00000000e+000]


C:\Users\Srishti Vishnoi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [30]:
len(ProbList)

10

In [12]:
ProbList

[array([0.00000000e+000, 3.06993295e-268, 7.78238865e-293, ...,
        5.37033759e-296, 6.70568144e-287, 0.00000000e+000]),
 array([0.00000000e+000, 0.00000000e+000, 3.00367383e-220, ...,
        5.45885389e-313, 2.29073043e-318, 0.00000000e+000]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        3.04695492e-296, 1.20343862e-315, 0.00000000e+000]),
 array([0.0000e+000, 0.0000e+000, 0.0000e+000, ..., 0.0000e+000,
        1.3196e-320, 0.0000e+000]),
 array([0.00000000e+000, 0.00000000e+000, 8.15504755e-320, ...,
        1.28185457e-313, 4.31220496e-320, 0.00000000e+000]),
 array([0.00000000e+000, 0.00000000e+000, 2.63032293e-279, ...,
        6.51617110e-306, 2.20845702e-296, 0.00000000e+000]),
 array([0.00000000e+000, 0.00000000e+000, 1.36998982e-283, ...,
        0.00000000e+000, 5.91381957e-291, 0.00000000e+000]),
 array([0.00000000e+000, 0.00000000e+000, 4.21941904e-307, ...,
        2.75028279e-300, 3.64783059e-312, 0.00

### Using SkLearn

#### Without Dimensionality 

In [20]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(X_Train_Data,Y_Train_Data)

predict_by_model = classifier.predict(X_Test_Data)


In [22]:
# for i in range(len(predict_by_model)):
#     print(predict_by_model[i],Y_Test_Data[i])

accuracy = sum(predict_by_model == Y_Test_Data)/(Y_Test_Data.shape[0])
print('Accuracy is: {}%'.format(accuracy*100))

Accuracy is: 10.77%


#### With Dimensionality Reduction 

In [23]:
X_Train_Data_PCA,X_Test_Data_PCA = PCA(X_Train_Data,X_Test_Data)

K : 330
LEN:  784
w.shape (784, 330)
X_Train_Data_PCA.shape (60000, 330)
X_Test_Data_PCA.shape (10000, 330)


In [24]:

classifier.fit(X_Train_Data_PCA,Y_Train_Data)

predict_by_model_PCA = classifier.predict(X_Test_Data_PCA)

In [25]:
accuracy = sum(predict_by_model_PCA == Y_Test_Data)/(Y_Test_Data.shape[0])
print('Accuracy is: {}%'.format(accuracy*100))

Accuracy is: 45.660000000000004%
